In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import joblib
from tqdm import tqdm
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score 
import joblib
import pickle
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/trainLabels_cropped.csv")
image_names = df['image']
y = df['level']
print(Counter(y))

# # C:\Users\jwei2\Bioe_485_final_project\archive\resized_train\resized_train\10_left.jpeg"
# images = []
# for image in tqdm(image_names):
#     img = cv2.imread(f'C:\\Users\\jwei2\\Bioe_485_final_project\\archive\\resized_train_cropped\\resized_train_cropped\\{image}.jpeg')
#     img = cv2.resize(img, (128,128))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     if img is not None:
#         images.append(img)

# X = np.array(images)
# joblib.dump(X, "X")

# print(X.shape)
# assert 1==2

X = joblib.load("/content/drive/MyDrive/X")
X = X.reshape( (X.shape[0], 128*128) )

oversample = SMOTE()
X_over, y_over = oversample.fit_resample(X, y)
for i in range(X_over.shape[0]):
  min = X_over[i].min(axis=0)
  max = X_over[i].max(axis=0)
  X_std = (X_over[i] - min) / (max - min)
  X_over[i] = X_std * (max - min) + min
print("oversampled: ",Counter(y_over))

indices = np.arange(X_over.shape[0])
train_val_indices, heldout_indices = train_test_split(indices, test_size=0.3, random_state=42, stratify=y_over)
X_train = X_over[train_val_indices]
y_train = y_over[train_val_indices]

clf = LogisticRegression(multi_class='multinomial', solver='saga')
clf.fit(X_train, y_train)

Counter({0: 25802, 2: 5288, 1: 2438, 3: 872, 4: 708})
oversampled:  Counter({0: 25802, 1: 25802, 2: 25802, 4: 25802, 3: 25802})


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(multi_class='multinomial', solver='saga')

In [ ]:
X_heldout = X_over[heldout_indices]
y_heldout = y_over[heldout_indices]

y_pred = clf.predict(X_heldout)
y_probs = clf.predict_proba(X_heldout)

In [ ]:
pickle.dump(clf, open( "/content/drive/MyDrive/logistic_regression.pkl", "wb" ))

print(classification_report(y_heldout, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.96      0.75      7741
           1       0.55      0.44      0.49      7740
           2       0.44      0.31      0.37      7741
           3       0.67      0.63      0.65      7741
           4       0.67      0.67      0.67      7740

    accuracy                           0.60     38703
   macro avg       0.59      0.60      0.58     38703
weighted avg       0.59      0.60      0.58     38703

